In [1]:
import pandas as pd
import os

In [2]:
import pickle

In [3]:
topics_df = pd.read_csv('../../../clean_data/all_topics.csv', index_col=0)

In [4]:
topics_df.head()

,goal_first_step,default_goal_type,is_questioned,are_first_steps_known,is_time_certain,is_certainly_imagined,are_obstackles_expected,goal_domain_key_pos,goal_name_key_pos,goal_weight,...,label_attractor_hard_skill,label_attractor_soft_skill,label_attractor_tool,label_attractor_community,label_attractor_subjectivity,label_attractor_habits,label_attractor_career,label_attractor_fixing,label_attractor_art,label_attractor_health
0,"Не знаю, с чего начать","Получить знания в новой области, Приобрести но...",0,0,0,1,0,творчество создание новый личный финансы работ...,отстать поезд современность,0.7017,...,1,1,0,1,0,1,1,1,0,0.0
1,"Не знаю, с чего начать","Получить знания в новой области, Приобрести но...",1,0,0,1,0,универсальный компетенция гибкий навык личный ...,самореализация,0.0082,...,1,0,1,0,1,0,1,0,1,0.0
2,Сменить работу,Включиться в новый проект/деятельность,0,1,0,1,0,иностранный язык гуманитарный знание навык ино...,образ жизнь,0.0431,...,1,1,0,0,0,0,0,1,0,0.0
3,поиск необходимой информации,Развить имеющиеся знания и навыки,0,1,0,1,0,прикладной знание навык ручной труд прикладной...,обновление устаревший информация,0.6316,...,1,0,0,0,1,1,1,1,1,0.0
4,С анализа,"Получить знания в новой области, Приобрести но...",0,1,0,1,0,универсальный компетенция гибкий навык творчес...,повышение результативность труд,0.1768,...,1,0,0,0,0,0,0,0,1,0.0


In [5]:
topics_df.columns

Index(['goal_first_step', 'default_goal_type', 'is_questioned',
       'are_first_steps_known', 'is_time_certain', 'is_certainly_imagined',
       'are_obstackles_expected', 'goal_domain_key_pos', 'goal_name_key_pos',
       'goal_weight', 'goal_words', 'goal_verbs_counter', 'goal_nouns_counter',
       'goal_numr_counter', 'goal_adj_counter', 'goal_digit_counter',
       'goal_aver_word_len', 'label_attractor_knowledge',
       'label_attractor_hard_skill', 'label_attractor_soft_skill',
       'label_attractor_tool', 'label_attractor_community',
       'label_attractor_subjectivity', 'label_attractor_habits',
       'label_attractor_career', 'label_attractor_fixing',
       'label_attractor_art', 'label_attractor_health'],
      dtype='object')

In [6]:
topics = topics_df.copy()

In [7]:
topic_cols = ['label_attractor_knowledge',
       'label_attractor_hard_skill', 'label_attractor_soft_skill',
       'label_attractor_tool', 'label_attractor_community',
       'label_attractor_subjectivity', 'label_attractor_habits',
       'label_attractor_career', 'label_attractor_fixing',
       'label_attractor_art', 'label_attractor_health']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

In [11]:
for topic in topic_cols:
    topics_text_vectors = topics[['goal_name_key_pos', topic]]
    topics_text_vectors.dropna(inplace=True)
    text = topics_text_vectors['goal_name_key_pos']
    y = topics_text_vectors[topic]
    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(text)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    xgb = XGBClassifier()
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    print(topic, acc, f1, roc_auc)

label_attractor_knowledge 0.8950162194043055 0.9445828144458281 0.5010790699400619
label_attractor_hard_skill 0.8112651135358301 0.8955272608553706 0.5025807430693561
label_attractor_soft_skill 0.7340017693895606 0.8461276015011941 0.4995141255795723
label_attractor_tool 0.9144795045709231 0.0 0.4998388136686009
label_attractor_community 0.8006487761722206 0.00879765395894428 0.5000462323904127
label_attractor_subjectivity 0.6293128870539664 0.08846990572878898 0.5018325561143536
label_attractor_habits 0.8849896785608965 0.005102040816326531 0.5003006973066852
label_attractor_career 0.6349159539958714 0.07611940298507462 0.4967641225975364
label_attractor_fixing 0.7502211736950752 0.02306805074971165 0.5011248947906334
label_attractor_art 0.9097611324093188 0.0 0.49983797796500323
label_attractor_health 0.9539958714243586 0.0 0.5


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
def create_model():
    vec_size = 100

    model = Sequential()
    model.add(Embedding(vocab_size, vec_size, input_length=max_len))

    model.add(Conv1D(32, 2, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(0.5))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(16, activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [15]:
results = []
for topic in topic_cols:
    topics_text_vectors = topics[['goal_name_key_pos', topic]]
    topics_text_vectors.dropna(inplace=True)
    text = topics_text_vectors['goal_name_key_pos']
    y = topics_text_vectors[topic]
    token = Tokenizer()
    token.fit_on_texts(text)
    vocab_size = len(token.word_index) + 1
    encoded_text = token.texts_to_sequences(text)
    max_len = int()
    for i in encoded_text:
        len_ = len(i)
        if len_ > max_len:
            max_len = len_
    X = pad_sequences(encoded_text, maxlen=max_len, padding='post')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
    model = create_model()
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train)
    X_test= np.asarray(X_test)
    y_test = np.asarray(y_test)
    model.fit(X_train, y_train, epochs=600, validation_data=(X_test, y_test), callbacks=[early_stop])
    y_pred = model.predict_classes(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    results.append((topic, acc, f1, roc_auc))

Train on 13562 samples, validate on 3391 samples
Epoch 1/600
13562/13562 [==============================] - 10s 758us/sample - loss: 0.3519 - accuracy: 0.8934 - val_loss: 0.4633 - val_accuracy: 0.8953
Epoch 2/600
13562/13562 [==============================] - 9s 677us/sample - loss: 0.3383 - accuracy: 0.8954 - val_loss: 0.4300 - val_accuracy: 0.8953
Epoch 3/600
13562/13562 [==============================] - 9s 665us/sample - loss: 0.3377 - accuracy: 0.8954 - val_loss: 0.4215 - val_accuracy: 0.8953
Epoch 4/600
13562/13562 [==============================] - 9s 651us/sample - loss: 0.3376 - accuracy: 0.8954 - val_loss: 0.4257 - val_accuracy: 0.8953
Epoch 5/600
13562/13562 [==============================] - 9s 654us/sample - loss: 0.3373 - accuracy: 0.8954 - val_loss: 0.4203 - val_accuracy: 0.8953
Epoch 6/600
13562/13562 [==============================] - 9s 664us/sample - loss: 0.3354 - accuracy: 0.8955 - val_loss: 0.4107 - val_accuracy: 0.8953
Epoch 7/600
13562/13562 [===================

13562/13562 [==============================] - 9s 687us/sample - loss: 0.3364 - accuracy: 0.8638 - val_loss: 0.7356 - val_accuracy: 0.7841
Epoch 22/600
13562/13562 [==============================] - 9s 689us/sample - loss: 0.3297 - accuracy: 0.8647 - val_loss: 0.7948 - val_accuracy: 0.7806
Epoch 23/600
13562/13562 [==============================] - 9s 685us/sample - loss: 0.3284 - accuracy: 0.8673 - val_loss: 0.7962 - val_accuracy: 0.7806
Epoch 24/600
13562/13562 [==============================] - 9s 681us/sample - loss: 0.3246 - accuracy: 0.8690 - val_loss: 0.7998 - val_accuracy: 0.7865
Epoch 25/600
13562/13562 [==============================] - 9s 682us/sample - loss: 0.3170 - accuracy: 0.8717 - val_loss: 0.8481 - val_accuracy: 0.7785
Epoch 26/600
13562/13562 [==============================] - 9s 679us/sample - loss: 0.3163 - accuracy: 0.8721 - val_loss: 0.8420 - val_accuracy: 0.7788.3159 - 
Epoch 27/600
13562/13562 [==============================] - 9s 678us/sample - loss: 0.3141 - 

13562/13562 [==============================] - 9s 695us/sample - loss: 0.2931 - accuracy: 0.9148 - val_loss: 0.4230 - val_accuracy: 0.9148
Epoch 7/600
13562/13562 [==============================] - 9s 687us/sample - loss: 0.2911 - accuracy: 0.9148 - val_loss: 0.4140 - val_accuracy: 0.9148
Epoch 8/600
13562/13562 [==============================] - 9s 693us/sample - loss: 0.2878 - accuracy: 0.9148 - val_loss: 0.4069 - val_accuracy: 0.9148
Epoch 9/600
13562/13562 [==============================] - 9s 694us/sample - loss: 0.2832 - accuracy: 0.9148 - val_loss: 0.3697 - val_accuracy: 0.9148
Epoch 10/600
13562/13562 [==============================] - 9s 691us/sample - loss: 0.2763 - accuracy: 0.9148 - val_loss: 0.3674 - val_accuracy: 0.9148
Epoch 11/600
13562/13562 [==============================] - 9s 689us/sample - loss: 0.2711 - accuracy: 0.9148 - val_loss: 0.3794 - val_accuracy: 0.9148
Epoch 12/600
13562/13562 [==============================] - 9s 688us/sample - loss: 0.2637 - accuracy: 0

KeyboardInterrupt: 

In [16]:
results

[('label_attractor_knowledge',
  0.86670598643468,
  0.9282539682539683,
  0.5026814377702314),
 ('label_attractor_hard_skill',
  0.7782365084046005,
  0.8738255033557045,
  0.49896712574753554),
 ('label_attractor_soft_skill',
  0.6968445886169271,
  0.8157706093189965,
  0.5030118795392682),
 ('label_attractor_tool',
  0.8956060159245061,
  0.03804347826086957,
  0.5005053107828636)]